In [12]:
import torch
from torchvision.transforms import v2

import sys
sys.path.append('src')
from src.models.frame import FrameModel

# ff_attribution -> 224
# swinv2_faceswap -> 256
rs_size = 256
interpolation = 3
inference_transforms = v2.Compose([
    v2.ToImage(),
    v2.Resize(rs_size, interpolation=interpolation, antialias=False),
    v2.ToDtype(torch.float32, scale=True),
    v2.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

In [13]:
# ff_attribution or swinv2_faceswap
model = FrameModel.load_from_checkpoint("./checkpoints/ff_attribution.ckpt").eval()

In [14]:
# (B, C, H, W)
test_image = torch.randn(1, 3, 224, 224).to(model.device)
test_image = inference_transforms(test_image)

with torch.no_grad():
    output = model(test_image)
    print(output)
    print(output.shape)

tensor([[0.6673, 0.2875, 0.0223, 0.0170, 0.0058]], device='cuda:1')
torch.Size([1, 5])
